In [10]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [11]:
from scipy.interpolate import interp1d
# resample target
def resample_cycle(cycle, num_points):
    """Resamples a gait cycle to have a fixed number of points using interpolation."""
    original_indices = np.linspace(0, 1, len(cycle))  # original indices based on the length of the cycle
    new_indices = np.linspace(0, 1, num_points)  # new indices to match the desired number of points
    interpolation_function = interp1d(original_indices, cycle, kind='linear')  # interpolate the data
    return interpolation_function(new_indices)  # return the resampled cycle

def rotate_cyclic(df, column_name, range):
    # Move the last value to the first position using pd.concat()
    return pd.concat([df[column_name].iloc[-range:], df[column_name].iloc[:-range]]).reset_index(drop=True)


In [12]:

leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_0.csv')
leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_1_v1.csv')
leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_2_new_v1.csv')

# leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_0.csv')
# leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_1.csv')
# leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_2.csv')

# leg_0_df.rename(columns={'tr': 'TR0', 'cr': 'CR0', 'fr': 'FR0', 'z': 'Z0'}, inplace=True)
# leg_1_df.rename(columns={'tr': 'TR1', 'cr': 'CR1', 'fr': 'FR1', 'z': 'Z1'}, inplace=True)
# leg_2_df.rename(columns={'tr': 'TR2', 'cr': 'CR2', 'fr': 'FR2', 'z': 'Z2'}, inplace=True)
# print(leg_0_df.columns)
# print(leg_1_df.columns)
# print(leg_2_df.columns)


# leg_0_df['TR0'] = rotate_cyclic(leg_0_df, 'TR0', 250)
# leg_0_df['CR0'] = rotate_cyclic(leg_0_df, 'CR0', 250)
# leg_0_df['FR0'] = rotate_cyclic(leg_0_df, 'FR0', 250)

# leg_1_df['TR1'] = rotate_cyclic(leg_1_df, 'TR1', 300)
# leg_1_df['CR1'] = rotate_cyclic(leg_1_df, 'CR1', 300)
# leg_1_df['FR1'] = rotate_cyclic(leg_1_df, 'FR1', 300)

# leg_2_df['TR2'] = rotate_cyclic(leg_2_df, 'TR2', 300)
# leg_2_df['CR2'] = rotate_cyclic(leg_2_df, 'CR2', 300)
# leg_2_df['FR2'] = rotate_cyclic(leg_2_df, 'FR2', 300)


fig = go.Figure()
fig = make_subplots(rows=4, cols=3, shared_xaxes=False)
fig.update_layout(height=1000, width=1100, title="Path plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=leg_0_df['TR0'], mode='lines', name=f'TR0'), row=1, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['CR0'], mode='lines', name=f'CR0'), row=2, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['FR0'], mode='lines', name=f'FR0'), row=3, col=1)
# fig.add_trace(go.Scatter( y=leg_0_df['Z0'], mode='lines', name=f'Z0'), row=4, col=1)

fig.add_trace(go.Scatter( y=leg_1_df['TR1'], mode='lines', name=f'TR1'), row=1, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['CR1'], mode='lines', name=f'CR1'), row=2, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['FR1'], mode='lines', name=f'FR1'), row=3, col=2)
# fig.add_trace(go.Scatter( y=leg_1_df['Z1'], mode='lines', name=f'Z1'), row=4, col=2)

fig.add_trace(go.Scatter( y=leg_2_df['TR2'], mode='lines', name=f'TR2'), row=1, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['CR2'], mode='lines', name=f'CR2'), row=2, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['FR2'], mode='lines', name=f'FR2'), row=3, col=3)
# fig.add_trace(go.Scatter( y=leg_2_df['Z2'], mode='lines', name=f'Z2'), row=4, col=3)

In [13]:
from include.cpg_rbf.cpg_so2 import CPG_SO2
# from include.cpg_rbf.cpg_afdc import CPG_AFDC
from include.cpg_rbf.rbf import RBF

rbf = RBF()
cpg = CPG_SO2()
cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
cpg_cycle_length = len(out0_cpg_one_cycle)
cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [14]:
columns = ['TR0', 'CR0', 'FR0']
# columns = ['TR0', 'CR0', 'FR0', 'Z0']
imitated_paths = {}
imitated_weights = {}
for col in columns:
    # Resample cycle for the current column
    target = resample_cycle(np.array(leg_0_df[col]), cpg_cycle_length)
    
    # Imitate path and get weights using RBF
    imitated_paths[col]     = rbf.imitate_path_by_learning(target)
    imitated_weights[col]   = rbf.get_rbf_weight()

columns = ['TR1', 'CR1', 'FR1']
# columns = ['TR1', 'CR1', 'FR1', 'Z1']
for col in columns:
    # Resample cycle for the current column
    target = resample_cycle(np.array(leg_1_df[col]), cpg_cycle_length)
    
    # Imitate path and get weights using RBF
    imitated_paths[col]     = rbf.imitate_path_by_learning(target)
    imitated_weights[col]   = rbf.get_rbf_weight()
    
columns = ['TR2', 'CR2', 'FR2']
# columns = ['TR2', 'CR2', 'FR2', 'Z2']
for col in columns:
    # Resample cycle for the current column
    target = resample_cycle(np.array(leg_2_df[col]), cpg_cycle_length)
    
    # Imitate path and get weights using RBF
    imitated_paths[col]     = rbf.imitate_path_by_learning(target)
    imitated_weights[col]   = rbf.get_rbf_weight()

last iteration: 11
last iteration: 9
last iteration: 17
last iteration: 13
last iteration: 16
last iteration: 44
last iteration: 13
last iteration: 14
last iteration: 16


In [15]:
np.savez('imitated_stick_insect_right_leg_weights.npz', **imitated_weights)

In [16]:
data = np.load('imitated_stick_insect_right_leg_weights.npz')
print(data.files)

# Access each array by its key
for key in data.files:
    print(f"{key}: {data[key]}")

['TR0', 'CR0', 'FR0', 'TR1', 'CR1', 'FR1', 'TR2', 'CR2', 'FR2']
TR0: [-0.01468128 -0.0032079   0.01855153  0.02928141  0.03684473  0.03931835
  0.04317047  0.04844651  0.05208282  0.05742784  0.06101039  0.06902041
  0.07480724  0.08346213  0.09109     0.09499901  0.09981778  0.10285264
  0.10822332  0.11215974  0.11770563  0.12016925  0.11780676  0.11553311
  0.11965701  0.12603504  0.12904808  0.13175207  0.13019318  0.12772221
  0.12462921  0.12155681  0.12035644  0.11664546  0.10987334  0.10339193
  0.09905742  0.09641865  0.09362756  0.0889534   0.08533549  0.07862123
  0.07061393  0.06425762  0.05422128  0.04696179  0.03413438  0.02324375
  0.00087191 -0.01303061]
CR0: [0.0359932  0.05633563 0.0825502  0.09286573 0.10130045 0.10474856
 0.10923307 0.11433154 0.11878545 0.12704987 0.1336201  0.14338151
 0.14959623 0.15523518 0.15570595 0.15389961 0.14991234 0.14423856
 0.13571065 0.13196932 0.12534637 0.11079655 0.09909212 0.0836731
 0.07794144 0.08586229 0.08582821 0.07847395 0.06

In [17]:

fig = go.Figure()
fig = make_subplots(rows=5, cols=1, shared_xaxes=False)
fig.update_layout(height=800, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# for i in range(len(cpg_kernel)):
#     fig.add_trace(go.Scatter( y=cpg_kernel[i], mode='lines', name=f'kernels{i}'), row=2, col=1)

fig.add_trace(go.Scatter( y=out0_cpg_one_cycle, mode='lines', name=f'out0_cpg_one_cycle'), row=1, col=1)
fig.add_trace(go.Scatter( y=out1_cpg_one_cycle, mode='lines', name=f'out1_cpg_one_cycle'), row=1, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['TR0']), cpg_cycle_length), mode='lines', name=f'TR0'), row=3, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['CR0']), cpg_cycle_length), mode='lines', name=f'CR0'), row=4, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=5, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['FR0']), cpg_cycle_length), mode='lines', name=f'FR0'), row=5, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['TR1'], mode='lines', name=f'TR1 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['TR1']), cpg_cycle_length), mode='lines', name=f'TR1'), row=3, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['CR1'], mode='lines', name=f'CR1 (CPG-RBF)'), row=4, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['CR1']), cpg_cycle_length), mode='lines', name=f'CR1'), row=4, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['FR1'], mode='lines', name=f'FR1 (CPG-RBF)'), row=5, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['FR1']), cpg_cycle_length), mode='lines', name=f'FR1'), row=5, col=1)

In [18]:
cpg = CPG_SO2()
cpg_output_0 = []
cpg_output_1 = []
online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[]}
for i in range(1000):
    cpg.s = 1.0
    cpg.update_cpg(0.05)      # phi would be equal when learning


    cpg_output_0.append(cpg.out0_t)
    cpg_output_1.append(cpg.out1_t)
    
    online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
    online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
    online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))

fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.update_layout(height=600, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)
